In [6]:
# 克隆posenet-python仓库
!git clone https://github.com/rwightman/posenet-python.git
%cd posenet-python

# 安装依赖
!pip install tensorflow==2.8.0 opencv-python
# 安装TensorFlow 2.8.0


Cloning into 'posenet-python'...
remote: Enumerating objects: 119, done.
remote: Total 119 (delta 0), reused 0 (delta 0), pack-reused 119
Receiving objects: 100% (119/119), 36.56 KiB | 634.00 KiB/s, done.
Resolving deltas: 100% (68/68), done.
/content/posenet-python/posenet-python/posenet-python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 46.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  At

In [ ]:
import tensorflow as tf
import posenet
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 加载模型
model = posenet.load_model(101)  # 使用ResNet50模型
model = tf.function(model)

# 上传图像
from google.colab import files
uploaded = files.upload()

# 处理上传的图像
image_path = list(uploaded.keys())[0]
img = cv2.imread(image_path)
input_img, draw_img, output_scale = posenet.process_input(img, scale_factor=0.7125, output_stride=16)

# 运行模型
heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = model(input_img)

# 解码结果
pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
    heatmaps_result.numpy().squeeze(axis=0),
    offsets_result.numpy().squeeze(axis=0),
    displacement_fwd_result.numpy().squeeze(axis=0),
    displacement_bwd_result.numpy().squeeze(axis=0),
    output_stride=16,
    max_pose_detections=5,
    min_pose_score=0.15)

keypoint_coords *= output_scale

# 绘制并显示结果
overlay_image = posenet.draw_skel_and_kp(
    draw_img, pose_scores, keypoint_scores, keypoint_coords,
    min_pose_score=0.15, min_part_score=0.1)

cv2_imshow(overlay_image)

In [ ]:
# 上传视频文件
uploaded_video = files.upload()
video_path = list(uploaded_video.keys())[0]

# 读取并处理视频
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_img, draw_img, output_scale = posenet.process_input(frame, scale_factor=0.7125, output_stride=16)
    heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = model(input_img)
    pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
        heatmaps_result.numpy().squeeze(axis=0),
        offsets_result.numpy().squeeze(axis=0),
        displacement_fwd_result.numpy().squeeze(axis=0),
        displacement_bwd_result.numpy().squeeze(axis=0),
        output_stride=16,
        max_pose_detections=5,
        min_pose_score=0.15)

    keypoint_coords *= output_scale
    overlay_image = posenet.draw_skel_and_kp(
        draw_img, pose_scores, keypoint_scores, keypoint_coords,
        min_pose_score=0.15, min_part_score=0.1)

    cv2_imshow(overlay_image)

cap.release()